In [1]:
import pandas as pd
import os
import shutil
from gwas.utils import shell_do
from gwas.qc import QC

In [2]:
# clone ILMN GTCtoVCF github repo
# !git clone https://github.com/Illumina/GTCtoVCF.git
# get hg38 reference
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.26_GRCh38/GCF_000001405.26_GRCh38_genomic.fna.gz -P ref_data
# !gunzip ref_data/GCF_000001405.26_GRCh38_genomic.fna.gz

In [3]:
basedir = '/data/CARD/PD/GP2/raw_genotypes'
out_genotypes = '/data/CARD/PD/GP2/genotypes'
shulman_ny_path = f'{basedir}/shulman_ny'
gtc_file_path = f'{shulman_ny_path}/GP2_GCT_files'
idat_file_path = f'{shulman_ny_path}/GP2_Shulman'
key_file = f'{gtc_file_path}/Key File_FINAL_Shulman_and_NY_011421.txt'
manifest_txt_path = f'{gtc_file_path}/FINALSS_after_rerun__Shulman_and_NY_011421.csv'
bpm = f'{gtc_file_path}/NeuroBooster_20042459_A1.bpm'
cluster_file = f'{gtc_file_path}/NBSCluster_file_n1393_011921.egt'

#software paths
GTCtoVCF = 'GTCtoVCF/gtc_to_vcf.py'
iaap = 'iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'

ref_fasta = 'ref_data/hg38_ref.fa'
shulman_gtc_path = f'{basedir}/SHULMAN/gtc_files'
shulman_idat_path = f'{basedir}/SHULMAN/idats'
shulman_out = f'{out_genotypes}/SHULMAN'
ny_gtc_path = f'{basedir}/NY/gtc_files'
ny_idat_path = f'{basedir}/NY/idats'
ny_out = f'{out_genotypes}/NY'


In [4]:
manifest = pd.read_csv(manifest_txt_path, header=10)
# key = pd.read_csv(key_file, sep='\t')

In [6]:
# create new directories to store split cohorts
!mkdir {basedir}/SHULMAN
!mkdir {basedir}/NY
!mkdir {basedir}/SHULMAN/gtc_files
!mkdir {basedir}/NY/gtc_files
!mkdir {basedir}/SHULMAN/idats
!mkdir {basedir}/NY/idats
!mkdir {out_genotypes}
!mkdir {out_genotypes}/SHULMAN
!mkdir {out_genotypes}/SHULMAN/vcfs
!mkdir {out_genotypes}/SHULMAN/iaap_called_gtcs
!mkdir {out_genotypes}/SHULMAN/ped
!mkdir {out_genotypes}/NY
!mkdir {out_genotypes}/NY/vcfs
!mkdir {out_genotypes}/NY/iaap_called_gtcs
!mkdir {out_genotypes}/NY/ped
!mkdir {shulman_out}/plink
!mkdir {ny_out}/plink


mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/SHULMAN': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/NY': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/SHULMAN/gtc_files': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/NY/gtc_files': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/raw_genotypes/NY/idats': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/SHULMAN': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/SHULMAN/vcfs': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/SHULMAN/iaap_called_gtcs': File exists
mkdir: cannot create directory '/data/CARD/PD/GP2/genotypes/SHULMAN/ped': File exists
mkdir: cannot create directory '/data/C

In [5]:
# create filenaames and split manifest into respective cohorts
manifest['filename'] = manifest['SentrixBarcode_A'].astype(str) + '_' + manifest['SentrixPosition_A']
shulman = manifest.loc[manifest.Study == 'Shulman']
ny = manifest.loc[manifest.Study == 'NY']

In [8]:
######### 1. IDAT TO PED/MAP STEPS #########

In [9]:
# create directory for each sentrix barcode A if they don't already exist-- 
for code in shulman.SentrixBarcode_A.unique():
    if os.path.exists(f'{basedir}/SHULMAN/idats/{code}'):
        print(f'{basedir}/SHULMAN/idats/{code} already exists')
    else:
        os.mkdir(f'{basedir}/SHULMAN/idats/{code}')

missing_idats = []

# split idats into respective cohorts and populated each sentrix barcode A directory
for i, filename in enumerate(shulman.filename):
    
    grn = f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat'
    red = f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat'

    if os.path.isfile(grn):
        shutil.copyfile(src=grn, dst=f'{basedir}/SHULMAN/idats/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat')
    else:
        missing_idats.append(grn)
        
    if os.path.isfile(red):
        shutil.copyfile(src=red, dst=f'{basedir}/SHULMAN/idats/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat')
    else:
        missing_idats.append(red)
    

len(missing_idats)

/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840024 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840019 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840027 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840026 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840003 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204701860066 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840025 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840002 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204701860124 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204701870098 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204701870028 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204697840010 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats/204701870016 already exists
/data/CARD/PD/GP2/raw_genotypes/SHULMA

0

In [10]:
# create directory for each sentrix barcode A if they don't already exist-- NY cohort
for code in ny.SentrixBarcode_A.unique():
    if os.path.exists(f'{basedir}/NY/idats/{code}'):
        print(f'{basedir}/NY/idats/{code} already exists')
    else:
        os.mkdir(f'{basedir}/NY/idats/{code}')
        
missing_idats = []
# spit idats into respective cohorts
for i, filename in enumerate(ny.filename):
    grn = f'{idat_file_path}/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat'
    red = f'{idat_file_path}/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat'
#     print(f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat')
#     print(f'{idat_file_path}/{shulman.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat')
    if os.path.isfile(grn):
        shutil.copyfile(src=grn, dst=f'{basedir}/NY/idats/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Grn.idat')
    else:
        missing_idats.append(grn)
        
    if os.path.isfile(red):
        shutil.copyfile(src=red, dst=f'{basedir}/NY/idats/{ny.SentrixBarcode_A.iloc[i]}/{filename}_Red.idat')
    else:
        missing_idats.append(red)
    

len(missing_idats)

/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701860128 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701860121 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870022 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870067 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870059 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870024 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701860122 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870060 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870018 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701860093 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870040 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870017 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870002 already exists
/data/CARD/PD/GP2/raw_genotypes/NY/idats/204701870029 already exists


0

In [11]:
# # swarm command to run in parallel
with open('shulman_idat_to_ped.swarm','w') as f:
    
    for code in shulman.SentrixBarcode_A.unique():
        
        shulman_idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{cluster_file} \
{shulman_out}/ped/ \
-f {shulman_idat_path}/{code} \
-p \
-t 8'
        
        f.write(f'{shulman_idat_to_ped_cmd}\n')
f.close()

In [26]:
# !swarm -f shulman_idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

In [15]:
# copy map file to match name of each ped
map = f'{shulman_out}/ped/NeuroBooster_20042459_A1.map'
for filename in shulman.filename:
    ped = f'{shulman_out}/ped/{filename}.ped'
    out_map = f'{shulman_out}/ped/{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')


In [9]:
for filename in shulman.filename:
    ped = f'{shulman_out}/ped/{filename}'
    make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {shulman_out}/plink/{filename}'

    shell_do(make_bed_cmd)

es/SHULMAN/ped/204701870118_R05C01 --make-bed --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/204701870118_R05C01
Executing: plink --file /data/CARD/PD/GP2/genotypes/SHULMAN/ped/204701870118_R06C01 --make-bed --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/204701870118_R06C01
Executing: plink --file /data/CARD/PD/GP2/genotypes/SHULMAN/ped/204701870118_R07C01 --make-bed --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/204701870118_R07C01
Executing: plink --file /data/CARD/PD/GP2/genotypes/SHULMAN/ped/204701870118_R08C01 --make-bed --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/204701870118_R08C01
Executing: plink --file /data/CARD/PD/GP2/genotypes/SHULMAN/ped/204701860019_R01C01 --make-bed --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/204701860019_R01C01
Executing: plink --file /data/CARD/PD/GP2/genotypes/SHULMAN/ped/204701860019_R02C01 --make-bed --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/204701860019_R02C01
Executing: plink --file /data/CARD/PD/GP2/genotypes/SHULMAN/ped/2047018

In [13]:
# write plink merge command
with open("shulman_merge_bed.list", 'w') as f:
    for filename in shulman.filename:
        bed = f'{shulman_out}/plink/{filename}'
        f.write(f'{bed}\n')
f.close()

with open("shulman_merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list shulman_merge_bed.list \
--make-bed \
--out {shulman_out}/plink/shulman'
    f.write(f"{plink_merge_cmd}")
f.close()
# shell_do(plink_merge_cmd, log=True, return_log=True)

In [14]:
!swarm -f shulman_merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

7619522


In [15]:
# get basic statistics
plink_miss_cmd = f'\
plink \
--bfile {shulman_out}/plink/shulman \
--missing \
--out {shulman_out}/plink/shulman'

shell_do(plink_miss_cmd)

Executing: plink --bfile /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman --missing --out /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman


In [23]:
# get average call rate
lmiss = pd.read_csv(f'{shulman_out}/plink/shulman.lmiss', sep='\s+')
imiss = pd.read_csv(f'{shulman_out}/plink/shulman.imiss', sep='\s+')
avg_call_rate = 100-lmiss.F_MISS.mean()
avg_geno_rate = 100-imiss.F_MISS.mean()
print(f'Average Call Rate: {avg_call_rate}')
print(f'Average Genotyping Rate: {avg_geno_rate}')

Average Call Rate: 99.98944379372823
Average Genotyping Rate: 99.98943549534161


In [6]:
# run sample-level QC
geno_name = f'{shulman_out}/plink/shulman'
geno_call_rate = geno_name + "_call_rate"
geno_call_rate_sex = geno_call_rate + "_sex"
# geno_het =  geno_sex + "_het"

# INSTANTIATE QC WITH INPUT NAME AND OUTPUT NAME     
qc = QC(geno_name, rare=False)

# NOW RUN COMMANDS
# FIRST, CLEAR EXISTING LOGFILE
qc.rm_log()

# run het pruning
qc.call_rate_pruning(geno_name)
qc.sex_check(geno_call_rate)
# qc.het_pruning(geno_sex)

PROCESSING THE FOLLOWING GENOTYPES: /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman

PRUNING FOR CALL RATE
logging to /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman.PLINK_STEPS.log
***********************************************

CHECKING SEXES
logging to /data/CARD/PD/GP2/genotypes/SHULMAN/plink/shulman.PLINK_STEPS.log
***********************************************



In [11]:
ref_dir_path = f'/data/LNG/vitaled2/1kgenomes'
ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel_gp2_pruned'
!python3 run_ancestry.py --geno {geno_call_rate_sex} --ref {ref_panel} --ref_labels {ref_dir_path}/gp2_ref_panel_labels.txt --out {shulman_out}/plink

807
  (0, 658)	0.22387749
  (0, 1080)	0.21524209
  (0, 1134)	0.24523933
  (0, 1318)	0.22929294
  (0, 1338)	0.22456564
  (0, 1352)	0.22713393
  (0, 1395)	0.3189124
  (0, 1495)	0.21364994
  (0, 1607)	0.20978947
  (0, 1612)	0.23310494
  (0, 1778)	0.22862315
  (0, 1895)	0.24963352
  (1, 127)	0.21575503
  (1, 164)	0.24361287
  (1, 252)	0.29668748
  (1, 616)	0.27403468
  (1, 631)	0.22343978
  :	:
  (474, 1513)	0.22756544
  (474, 1650)	0.23994713
  (474, 1782)	0.22530544
  (474, 1895)	0.23169318
  (474, 1902)	0.22799486
  (475, 61)	0.21913543
  (475, 294)	0.2355824
  (475, 300)	0.32018295
  (475, 322)	0.2317408
  (475, 405)	0.24028732
  (475, 421)	0.25017998
  (475, 519)	0.2218786
  (475, 537)	0.20684372
  (475, 592)	0.21839552
  (475, 725)	0.19622323
  (475, 784)	0.20143083
  (475, 787)	0.21256885
  (475, 1032)	0.23543279
  (475, 1278)	0.22859171
  (475, 1300)	0.2299052
  (475, 1473)	0.21593755
  (475, 1615)	0.22935642
  (475, 1737)	0.22001056
  (475, 1808)	0.21328261
  (475, 1860)	0.3151493

Python 3.6.10 :: Anaconda, Inc.


In [22]:
# copy gtc files to respective directories
# for filename in shulman.filename:
#     shutil.copyfile(src=f'{gtc_file_path}/{filename}.gtc',dst=f'{basedir}/SHULMAN/gtc_files/{filename}.gtc')
# for filename in ny.filename:
#     shutil.copyfile(src=f'{gtc_file_path}/{filename}.gtc',dst=f'{basedir}/NY/gtc_files/{filename}.gtc')  

/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R01C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R02C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R03C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R04C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R05C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R06C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R07C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840024_R08C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840019_R01C01
/data/CARD/PD/GP2/genotypes/SHULMAN/ped/204697840019_R02C01


In [8]:
# swarm command to run in parallel
with open('shulman_gtc_to_vcf.swarm','w') as f:
    
    for filename in shulman.filename:
        
        shulman_gtc_to_vcf_cmd = f'\
python3 {GTCtoVCF} \
--gtc-paths {shulman_gtc_path}/{filename}.gtc \
--manifest-file {bpm} \
--genome-fasta-file {ref_fasta} \
--output-vcf-path {shulman_out}/vcfs/{filename}.vcf \
--skip-indels'
        
        f.write(f'{shulman_gtc_to_vcf_cmd}\n')
f.close()


In [9]:
# !swarm -f shulman_gtc_to_vcf.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

In [10]:
# swarm command to run in parallel
with open('ny_gtc_to_vcf.swarm','w') as f:
    
    for filename in ny.filename:
        
        ny_gtc_to_vcf_cmd = f'\
python3 {GTCtoVCF} \
--gtc-paths {ny_gtc_path}/{filename}.gtc \
--manifest-file {bpm} \
--genome-fasta-file {ref_fasta} \
--output-vcf-path {ny_out}/vcfs/{filename}.vcf \
--skip-indels'
        
        f.write(f'{ny_gtc_to_vcf_cmd}\n')
f.close()


In [24]:
# !swarm -f ny_gtc_to_vcf.swarm -g 16 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

6960101


7424698


In [39]:
### PROBABLY NOT GOING TO USE THIS

# use picard to merge vcfs
# create vcf_list
with open('vcf.list','w') as f:
    
    for filename in shulman.filename:
        
        f.write(f'{shulman_out}/vcfs/{filename}.vcf\n')

f.close()

# write picard batch job
with open('merge_vcfs.sh','w') as f:
    f.write(f'#!/bin/bash\n\
set -e\n\
module load picard\n\
java -Xmx4g -XX:ParallelGCThreads=16 -jar $PICARDJARPATH/picard.jar CombineGenotypingArrayVcfs I=vcf.list O={shulman_out}/vcfs/shulman_merged.vcf')
f.close()

In [40]:
# !swarm -f shulman_vcf_to_bed.swarm -g 32 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --module plink --partition=norm
# !sbatch --cpus-per-task=32 merge_vcfs.sh

7440750


In [50]:
!ls {shulman_out}/vcfs/shulman_merged.vcf.*

/data/CARD/PD/GP2/genotypes/SHULMAN/vcfs/shulman_merged.vcf.idx


In [46]:

##### PROBABLY NOT GOING TO USE THIS!!!
# # swarm command to run in parallel
# with open('shulman_idat_to_gtc.swarm','w') as f:
    
#     for code in shulman.SentrixBarcode_A.unique():
        
#         shulman_idat_to_gtc_cmd = f'\
# {iaap} gencall \
# {bpm} \
# {cluster_file} \
# {shulman_out}/iaap_called_gtcs/ \
# -f {shulman_idat_path}/{code} \
# -g \
# -t 16'
        
#         f.write(f'{shulman_idat_to_gtc_cmd}\n')
# f.close()


In [47]:
# !swarm -f shulman_idat_to_gtc.swarm -g 32 -t 16 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

7446145


7478742


In [17]:
shulman_idat_path

'/data/CARD/PD/GP2/raw_genotypes/SHULMAN/idats'

In [18]:
!cat shulman_idat_to_ped.swarm | wc -l

163
